In [81]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [82]:
product_type = "ambient"
num_time_periods = 365

In [83]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
#article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]
article_data = article_data.head(10)

In [84]:

if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
max_order_exceed_multiplier = 1.5

In [85]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [86]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [87]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [88]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [89]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [90]:
def getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - cum_demand > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - cum_demand) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost 
    return max_order 

In [91]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0

    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue
            cum_demand += item_demand[t]
            
            if(t==order_date+life):
                orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) 
                tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item) 
            if(math.ceil(cum_demand/cu_per_tu_item) > maximum_order_quantity_tu_item):
                #penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier * variable_cost
                penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier  
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    
                    orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                    #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item,variable_cost, maximum_order_quantity_tu_item)
                    tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                    tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                    tot_fixed += fixed_cost   
                    tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                 
                    run_complete = True
                    break

            else:
                orders[order_date] = math.ceil((cum_demand- item_demand[t])/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                tot_ordering += math.ceil((cum_demand- item_demand[t])/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)

                order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
        
    
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


35815.80000000001
Ordering 30661.600000000002
Fixing  5133.200000000004
Clearance  21.000000000000004
Max Order  0


In [92]:
buf_cost = 0
vol = 0
for time in range(len(time_indexes)):
    if time in schedule.keys():
        for item in schedule[time].keys():
            vol += schedule[time][item] * cu_per_tu[item] * volume_per_cu[item]
    
    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if vol > warehouse_volume:
        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost

0

In [93]:
obj_val + buf_cost

35815.80000000001

In [94]:
shelf_life

{'85237ed8': 1000,
 '6e88489e': 1000,
 'fb2957a9': 1000,
 '4c0bb0eb': 1000,
 '9b3d9b18': 1000,
 'b8a87ab3': 1000,
 '7a948891': 1000,
 'f267d4ff': 1000,
 '71849e26': 1000,
 '7afba9c4': 1000}

In [95]:
schedule

{'4c0bb0eb': {0: 4,
  4: 5,
  9: 9,
  17: 7,
  24: 7,
  31: 6,
  36: 7,
  43: 13,
  52: 13,
  66: 14,
  81: 5,
  87: 6,
  94: 9,
  108: 11,
  115: 5,
  121: 7,
  127: 10,
  141: 4,
  148: 3,
  155: 1,
  157: 7,
  170: 6,
  179: 1,
  183: 4,
  192: 4,
  198: 12,
  212: 9,
  225: 2,
  227: 7,
  233: 10,
  247: 4,
  255: 5,
  263: 2,
  269: 8,
  281: 6,
  290: 9,
  303: 7,
  310: 5,
  316: 10,
  326: 8,
  338: 5,
  346: 2,
  351: 4,
  359: 4},
 '6e88489e': {0: 80,
  3: 173,
  8: 290,
  17: 287,
  24: 226,
  30: 201,
  36: 77,
  38: 168,
  43: 117,
  46: 195,
  52: 252,
  59: 156,
  64: 354,
  72: 206,
  78: 124,
  80: 470,
  86: 371,
  92: 155,
  99: 69,
  101: 79,
  103: 59,
  106: 163,
  114: 137,
  142: 20,
  144: 45,
  151: 37,
  155: 214,
  162: 79,
  165: 139,
  169: 165,
  176: 140,
  183: 198,
  190: 217,
  218: 83,
  358: 94,
  361: 52},
 '71849e26': {0: 1,
  9: 1,
  11: 1,
  17: 1,
  19: 1,
  24: 2,
  29: 1,
  33: 1,
  36: 2,
  43: 1,
  47: 1,
  51: 1,
  53: 1,
  59: 2,
  65: 3,

In [96]:
# Modified schedule
modified_schedule = {}

# Iterate over each article ID
for article_id, orders in schedule.items():
    modified_orders = {}
    ordered_days = sorted(orders.keys())  # Sort the order days in ascending order
    
    # Iterate over pairs of consecutive days
    for i in range(0, len(ordered_days), 2):
        day1 = ordered_days[i]
        if i + 1 < len(ordered_days):
            day2 = ordered_days[i + 1]
        else:
            # If odd number of days, leave the last day as it is
            modified_orders[day1] = orders[day1]
            break

        # Calculate the duration between the two days
        duration = day2 - day1
        
        # Check if the duration is less than the shelf life
        if duration < shelf_life[article_id]:
            # Merge the orders of both days into the first day
            modified_orders[day1] = orders[day1] + orders[day2]
        else:
            # Place orders on their respective days
            modified_orders[day1] = orders[day1]
            modified_orders[day2] = orders[day2]

    modified_schedule[article_id] = modified_orders

# Print the modified schedule
print(modified_schedule)

{'4c0bb0eb': {0: 9, 9: 16, 24: 13, 36: 20, 52: 27, 81: 11, 94: 20, 115: 12, 127: 14, 148: 4, 157: 13, 179: 5, 192: 16, 212: 11, 227: 17, 247: 9, 263: 10, 281: 15, 303: 12, 316: 18, 338: 7, 351: 8}, '6e88489e': {0: 253, 8: 577, 24: 427, 36: 245, 43: 312, 52: 408, 64: 560, 78: 594, 86: 526, 99: 148, 103: 222, 114: 157, 144: 82, 155: 293, 165: 304, 176: 338, 190: 300, 358: 146}, '71849e26': {0: 2, 11: 2, 19: 3, 29: 2, 36: 3, 47: 2, 53: 3, 65: 4, 74: 2, 80: 2, 87: 3, 98: 2, 109: 2, 122: 4, 134: 2, 144: 2, 158: 3, 172: 2, 183: 2, 193: 2, 200: 3, 213: 2, 225: 2, 235: 2, 242: 2, 254: 2, 262: 1}, '7a948891': {85: 4, 99: 9, 114: 3, 136: 2, 151: 2, 210: 2, 270: 2, 294: 2, 336: 2, 353: 1}, '7afba9c4': {0: 2, 10: 3, 25: 4, 43: 2, 53: 3, 65: 3, 79: 4, 92: 2, 103: 3, 122: 3, 134: 2, 145: 2, 156: 2, 168: 2, 176: 4, 192: 3, 204: 5, 222: 2, 238: 4}, '85237ed8': {0: 8, 9: 9, 22: 16, 38: 14, 54: 3, 60: 4, 67: 7, 78: 9, 92: 6, 100: 6, 114: 9, 128: 5, 136: 7, 145: 4, 151: 9, 163: 10, 176: 4, 185: 5, 193: 3